## Homework 02: Unsupervised embedding-based MT
*Note: this homework is based on open materials from yandexdataschool [NLP course](https://github.com/yandexdataschool/nlp_course/). Feel free to check this awesome course if you wish to dig deeper.*

*Refined by [Nikolay Karpachev](https://www.linkedin.com/in/nikolay-karpachev-b0146a104/)*

**In this homework** **<font color='red'>YOU</font>** will make machine translation system without using parallel corpora, alignment, attention, 100500 depth super-cool recurrent neural network and all that kind superstuff.

But even without parallel corpora this system can be good enough (hopefully), in particular for similar languages, e.g. Ukrainian and Russian. 

### Frament of the Swadesh list for some slavic languages

The Swadesh list is a lexicostatistical stuff. It's named after American linguist Morris Swadesh and contains basic lexis. This list are used to define subgroupings of languages, its relatedness.

So we can see some kind of word invariance for different Slavic languages.


| Russian         | Belorussian              | Ukrainian               | Polish             | Czech                         | Bulgarian            |
|-----------------|--------------------------|-------------------------|--------------------|-------------------------------|-----------------------|
| женщина         | жанчына, кабета, баба    | жінка                   | kobieta            | žena                          | жена                  |
| мужчина         | мужчына                  | чоловік, мужчина        | mężczyzna          | muž                           | мъж                   |
| человек         | чалавек                  | людина, чоловік         | człowiek           | člověk                        | човек                 |
| ребёнок, дитя   | дзіця, дзіцёнак, немаўля | дитина, дитя            | dziecko            | dítě                          | дете                  |
| жена            | жонка                    | дружина, жінка          | żona               | žena, manželka, choť          | съпруга, жена         |
| муж             | муж, гаспадар            | чоловiк, муж            | mąż                | muž, manžel, choť             | съпруг, мъж           |
| мать, мама      | маці, матка              | мати, матір, неня, мама | matka              | matka, máma, 'стар.' mateř    | майка                 |
| отец, тятя      | бацька, тата             | батько, тато, татусь    | ojciec             | otec                          | баща, татко           |
| много           | шмат, багата             | багато                  | wiele              | mnoho, hodně                  | много                 |
| несколько       | некалькі, колькі         | декілька, кілька        | kilka              | několik, pár, trocha          | няколко               |
| другой, иной    | іншы                     | інший                   | inny               | druhý, jiný                   | друг                  |
| зверь, животное | жывёла, звер, істота     | тварина, звір           | zwierzę            | zvíře                         | животно               |
| рыба            | рыба                     | риба                    | ryba               | ryba                          | риба                  |
| птица           | птушка                   | птах, птиця             | ptak               | pták                          | птица                 |
| собака, пёс     | сабака                   | собака, пес             | pies               | pes                           | куче, пес             |
| вошь            | вош                      | воша                    | wesz               | veš                           | въшка                 |
| змея, гад       | змяя                     | змія, гад               | wąż                | had                           | змия                  |
| червь, червяк   | чарвяк                   | хробак, черв'як         | robak              | červ                          | червей                |
| дерево          | дрэва                    | дерево                  | drzewo             | strom, dřevo                  | дърво                 |
| лес             | лес                      | ліс                     | las                | les                           | гора, лес             |
| палка           | кій, палка               | палиця                  | patyk, pręt, pałka | hůl, klacek, prut, kůl, pálka | палка, пръчка, бастун |

But the context distribution of these languages demonstrates even more invariance. And we can use this fact for our for our purposes.

## Data

In [31]:
import textdistance
import gensim
import numpy as np
from gensim.models import KeyedVectors

In this notebook we're going to use pretrained word vectors - FastText (original paper - https://arxiv.org/abs/1607.04606).

You can download them from the official [website](https://fasttext.cc/docs/en/crawl-vectors.html). We're going to need embeddings for Russian and Ukrainian languages. Please use word2vec-compatible format (.text).

In [2]:
uk_emb = KeyedVectors.load_word2vec_format("./pretrained_emds/cc.uk.300.vec")

In [3]:
ru_emb = KeyedVectors.load_word2vec_format("./pretrained_emds/cc.ru.300.vec")

In [4]:
ru_emb.most_similar([ru_emb["август"]], topn=10)

[('август', 1.0),
 ('июль', 0.9383153915405273),
 ('сентябрь', 0.9240028858184814),
 ('июнь', 0.9222575426101685),
 ('октябрь', 0.9095538854598999),
 ('ноябрь', 0.8930036425590515),
 ('апрель', 0.8729087114334106),
 ('декабрь', 0.8652557730674744),
 ('март', 0.8545796275138855),
 ('февраль', 0.8401416540145874)]

In [5]:
uk_emb.most_similar([uk_emb["серпень"]])

[('серпень', 0.9999999403953552),
 ('липень', 0.9096440076828003),
 ('вересень', 0.901697039604187),
 ('червень', 0.8992519378662109),
 ('жовтень', 0.8810408711433411),
 ('листопад', 0.8787633776664734),
 ('квітень', 0.8592804670333862),
 ('грудень', 0.8586863279342651),
 ('травень', 0.8408110737800598),
 ('лютий', 0.8256431818008423)]

In [6]:
ru_emb.most_similar([uk_emb["серпень"]])

[('Stepashka.com', 0.2757962942123413),
 ('ЖИЗНИВадим', 0.25203436613082886),
 ('2Дмитрий', 0.25048112869262695),
 ('2012Дмитрий', 0.24829231202602386),
 ('Ведущий-Алексей', 0.2443869560956955),
 ('Недопустимость', 0.24435284733772278),
 ('2Михаил', 0.23981399834156036),
 ('лексей', 0.23740756511688232),
 ('комплексн', 0.23695150017738342),
 ('персональ', 0.2368222028017044)]

Load small dictionaries for correspoinding words pairs as trainset and testset.

In [7]:
def load_word_pairs(filename):
    uk_ru_pairs = []
    uk_vectors = []
    ru_vectors = []
    with open(filename, "r") as inpf:
        for line in inpf:
            uk, ru = line.rstrip().split("\t")
            if uk not in uk_emb or ru not in ru_emb:
                continue
            uk_ru_pairs.append((uk, ru))
            uk_vectors.append(uk_emb[uk])
            ru_vectors.append(ru_emb[ru])
    return uk_ru_pairs, np.array(uk_vectors), np.array(ru_vectors)

In [8]:
!wget -O ukr_rus.train.txt http://tiny.cc/jfgecz

--2021-04-18 17:50:03--  http://tiny.cc/jfgecz
Resolving tiny.cc (tiny.cc)... 157.245.113.153
Connecting to tiny.cc (tiny.cc)|157.245.113.153|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://tiny.cc/jfgecz [following]
--2021-04-18 17:50:03--  https://tiny.cc/jfgecz
Connecting to tiny.cc (tiny.cc)|157.245.113.153|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://raw.githubusercontent.com/yandexdataschool/nlp_course/master/week01_embeddings/ukr_rus.train.txt [following]
--2021-04-18 17:50:04--  https://raw.githubusercontent.com/yandexdataschool/nlp_course/master/week01_embeddings/ukr_rus.train.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 59351 (58K) [text/plain]
Sa

In [9]:
!wget -O ukr_rus.test.txt http://tiny.cc/6zoeez

--2021-04-18 17:50:04--  http://tiny.cc/6zoeez
Resolving tiny.cc (tiny.cc)... 157.245.113.153
Connecting to tiny.cc (tiny.cc)|157.245.113.153|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://tiny.cc/6zoeez [following]
--2021-04-18 17:50:04--  https://tiny.cc/6zoeez
Connecting to tiny.cc (tiny.cc)|157.245.113.153|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://raw.githubusercontent.com/yandexdataschool/nlp_course/master/week01_embeddings/ukr_rus.test.txt [following]
--2021-04-18 17:50:05--  https://raw.githubusercontent.com/yandexdataschool/nlp_course/master/week01_embeddings/ukr_rus.test.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12188 (12K) [text/plain]
Savi

In [10]:
uk_ru_train, X_train, Y_train = load_word_pairs("ukr_rus.train.txt")

In [11]:
uk_ru_test, X_test, Y_test = load_word_pairs("ukr_rus.test.txt")

## Embedding space mapping (0.3 pts)

Let $x_i \in \mathrm{R}^d$ be the distributed representation of word $i$ in the source language, and $y_i \in \mathrm{R}^d$ is the vector representation of its translation. Our purpose is to learn such linear transform $W$ that minimizes euclidian distance between $Wx_i$ and $y_i$ for some subset of word embeddings. Thus we can formulate so-called Procrustes problem:

$$W^*= \arg\min_W \sum_{i=1}^n||Wx_i - y_i||_2$$
or
$$W^*= \arg\min_W ||WX - Y||_F$$

where $||*||_F$ - Frobenius norm.

$W^*= \arg\min_W \sum_{i=1}^n||Wx_i - y_i||_2$ looks like simple multiple linear regression (without intercept fit). So let's code.

In [12]:
from sklearn.linear_model import LinearRegression

mapping = LinearRegression(fit_intercept=False)
mapping.fit(X_train, Y_train)

LinearRegression(fit_intercept=False)

Let's take a look at neigbours of the vector of word _"серпень"_ (_"август"_ in Russian) after linear transform.

In [13]:
august = mapping.predict(uk_emb["серпень"].reshape(1, -1))
ru_emb.most_similar(august)

[('апрель', 0.8531433343887329),
 ('июнь', 0.8402522206306458),
 ('март', 0.8385883569717407),
 ('сентябрь', 0.8331484198570251),
 ('февраль', 0.8311208486557007),
 ('октябрь', 0.8278018832206726),
 ('ноябрь', 0.8243727684020996),
 ('июль', 0.822961688041687),
 ('август', 0.8112279176712036),
 ('январь', 0.8022985458374023)]

We can see that neighbourhood of this embedding cosists of different months, but right variant is on the ninth place.

As quality measure we will use precision top-1, top-5 and top-10 (for each transformed Ukrainian embedding we count how many right target pairs are found in top N nearest neighbours in Russian embedding space).

In [14]:
def precision(pairs, mapped_vectors, topn=1):
    """
    :args:
        pairs = list of right word pairs [(uk_word_0, ru_word_0), ...]
        mapped_vectors = list of embeddings after mapping from source embedding space to destination embedding space
        topn = the number of nearest neighbours in destination embedding space to choose from
    :returns:
        precision_val, float number, total number of words for those we can find right translation at top K.
    """
    assert len(pairs) == len(mapped_vectors)
    num_matches = 0
    for i, (_, ru) in enumerate(pairs):
        if ru in set(map(lambda x: x[0], ru_emb.most_similar([mapped_vectors[i]], topn=topn))):
            num_matches += 1
    precision_val = num_matches / len(pairs)
    return precision_val

In [15]:
assert precision([("серпень", "август")], august, topn=5) == 0.0
assert precision([("серпень", "август")], august, topn=9) == 1.0
assert precision([("серпень", "август")], august, topn=10) == 1.0

In [16]:
assert precision(uk_ru_test, X_test) == 0.0
assert precision(uk_ru_test, Y_test) == 1.0

In [17]:
precision_top1 = precision(uk_ru_test, mapping.predict(X_test), 1)
precision_top5 = precision(uk_ru_test, mapping.predict(X_test), 5)

In [18]:
print(precision_top1)
print(precision_top5)

0.628498727735369
0.7913486005089059


## Making it better (orthogonal Procrustean problem) (0.3 pts)

It can be shown (see original paper) that a self-consistent linear mapping between semantic spaces should be orthogonal. 
We can restrict transform $W$ to be orthogonal. Then we will solve next problem:

$$W^*= \arg\min_W ||WX - Y||_F \text{, where: } W^TW = I$$

$$I \text{- identity matrix}$$

Instead of making yet another regression problem we can find optimal orthogonal transformation using singular value decomposition. It turns out that optimal transformation $W^*$ can be expressed via SVD components:
$$X^TY=U\Sigma V^T\text{, singular value decompostion}$$
$$W^*=UV^T$$

In [19]:
import numpy as np

In [20]:
def learn_transform(X_train, Y_train):
    """ 
    :returns: W* : float matrix[emb_dim x emb_dim] as defined in formulae above
    """
    u, s, vh = np.linalg.svd(X_train.T @ Y_train, full_matrices=False)
    mapping = u @ vh
    return mapping

In [21]:
W = learn_transform(X_train, Y_train)

In [22]:
ru_emb.most_similar([np.matmul(uk_emb["серпень"], W)])

[('апрель', 0.8245131969451904),
 ('июнь', 0.8056628704071045),
 ('сентябрь', 0.8055762052536011),
 ('март', 0.8032935261726379),
 ('октябрь', 0.798710286617279),
 ('июль', 0.7946796417236328),
 ('ноябрь', 0.7939636707305908),
 ('август', 0.7938189506530762),
 ('февраль', 0.7923861742019653),
 ('декабрь', 0.7715376615524292)]

In [23]:
print(precision(uk_ru_test, np.matmul(X_test, W)))
print(precision(uk_ru_test, np.matmul(X_test, W), 5))

0.6437659033078881
0.7989821882951654


## Unsupervised embedding-based MT (0.4 pts)

Now, let's build our word embeddings-based translator!

Firstly, download OPUS Tatoeba corpus.

In [24]:
!wget https://object.pouta.csc.fi/OPUS-Tatoeba/v20190709/mono/uk.txt.gz

--2021-04-18 17:53:05--  https://object.pouta.csc.fi/OPUS-Tatoeba/v20190709/mono/uk.txt.gz
Resolving object.pouta.csc.fi (object.pouta.csc.fi)... 86.50.254.18, 86.50.254.19
Connecting to object.pouta.csc.fi (object.pouta.csc.fi)|86.50.254.18|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1819128 (1,7M) [application/gzip]
Saving to: ‘uk.txt.gz’

uk.txt.gz           100%[===================>]   1,73M  6,05MB/s    in 0,3s    

2021-04-18 17:53:06 (6,05 MB/s) - ‘uk.txt.gz’ saved [1819128/1819128]



In [25]:
!gzip -d ./uk.txt.gz

gzip: ./uk.txt already exists; do you wish to overwrite (y or n)? ^C


In [26]:
with open('./uk.txt', 'r') as f:
    uk_corpus = f.readlines()

In [27]:
# To save your time and CPU, feel free to use first 1000 sentences of the corpus
uk_corpus = uk_corpus[:1000]

In [45]:
from multiprocessing import Pool, cpu_count

In [58]:
def similarity(a_b):
    return textdistance.levenshtein.normalized_similarity(a_b[0], a_b[1])

def translate(sentence):
    """
    :args:
        sentence - sentence in Ukrainian (str)
    :returns:
        translation - sentence in Russian (str)

    * find ukrainian embedding for each word in sentence
    * transform ukrainian embedding vector
    * find nearest russian word and replace
    """
    wv_words = uk_emb.index2word
    tokens = sentence.split()
    uk_embds = []
    for tok in tokens:
        try:
            uk_embds.append(uk_emb.get_vector(tok))
        except KeyError as e:
            with Pool(cpu_count() // 2) as pool:
                similarities = pool.map(similarity, zip(wv_words, [tok for _ in range(len(wv_words))]))
            uk_embds.append(uk_emb.get_vector(wv_words[np.argmax(similarities)]))
    uk_embds = np.array(uk_embds)
    
    transformed_embds = uk_embds @ W
    translated = list(map(lambda x: ru_emb.most_similar([x], topn=1)[0][0], transformed_embds))
    return " ".join(translated)

In [59]:
assert translate(".") == "."
assert translate("1 , 3") == "1 , 3"
assert translate("кіт зловив мишу") == "кот поймал мышку"

Now you can play with your model and try to get as accurate translations as possible. **Note**: one big issue is out-of-vocabulary words. Try to think of various ways of handling it (you can start with translating each of them to a special **UNK** token and then move to more sophisticated approaches). Good luck!

In [60]:
for sent in uk_corpus[::10]:
    print(translate(sent))

Я уже закончу колледж когда мы прибежишь со океании
Город бомбили враждебные самолеты
Возможно мной антисоциальный конечно это не означает что мной не общаюсь со людьми
Впрочем утра выпала рус.
Беда не приходит одна
Посмотри по тот жеребей
Я заказал два гамбургера
Я не хотел никого обидеть
Гора покрыта снегом
по фотографии во девушки корона не со золота а со цветы
Во меня То мечта
Я приехал во Японию со Китая
по север находится Шотландия по юге — Англия по востоке — Уэльса и ещe дальше по востоке — северная Ирландия
Его родная страна — Германия
Берн — столица Швейцарии
Он ждал по него к десятой часа
Ты можешь взять ту книгу даром
Такой роман сочинил известный американский писатель
забронировать будте ласковые комнату возле международного аэропорта во Торонто
Он знает что ты его влюбится
Я знаю что ты богатый
Те кто всё забывают счастливые
Во этой реке опасно плавать
пришел увидел победил
Я хожу к школы пешком
Не моя дело
Не забудь билет
Кто он
Вы будете чай ли кофе
Он не пойдет по пикн